# Copy Mechanism

## CopyNet 
![CopyNet](./imgs/copynet.png)


![Summarization](./imgs/summarization_copy.png)

[1] Gu, Jiatao, Zhengdong Lu, Hang Li, and Victor O.K. Li. “Incorporating Copying Mechanism in Sequence-to-Sequence Learning.” In Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), 1631–40. Berlin, Germany: Association for Computational Linguistics, 2016. https://doi.org/10.18653/v1/P16-1154.

[2] See, Abigail, Peter J. Liu, and Christopher D. Manning. “Get To The Point: Summarization with Pointer-Generator Networks.” In Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), 1073–83. Vancouver, Canada: Association for Computational Linguistics, 2017. https://doi.org/10.18653/v1/P17-1099.



In [1]:
import os,sys 
sys.path.append("..")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from transformers import BartTokenizer, BartForConditionalGeneration
from copy_mechanism import BartConditionalGenerationWithCopy

In [15]:
# tokenizer = BartTokenizer.from_pretrained("/pretrains/pt/facebook-bart-base")
# raw_model = BartForConditionalGeneration.from_pretrained("/pretrains/pt/facebook-bart-base")
# copy_model = BartConditionalGenerationWithCopy.from_pretrained("/pretrains/pt/facebook-bart-base")


tokenizer = BartTokenizer.from_pretrained("/pretrains/pt/bart_nl2sparql")
raw_model = BartForConditionalGeneration.from_pretrained("/pretrains/pt/bart_nl2sparql")
copy_model = BartConditionalGenerationWithCopy.from_pretrained("/pretrains/pt/bart_nl2sparql")


Some weights of BartConditionalGenerationWithCopy were not initialized from the model checkpoint at /pretrains/pt/bart_nl2sparql and are newly initialized: ['copy_linear.2.bias', 'copy_linear.0.weight', 'copy_linear.2.weight', 'copy_linear.0.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Raw Bart Model Demo

In [18]:
# """
# SELECT DISTINCT ?qpv WHERE { 
#     ?e <pred:instance_of> ?c . ?c <pred:name> \"big city\" . ?e <postal_code> ?pv_1 . ?pv_1 <pred:value> \"54000\" . 
#     ?e <population> ?pv . ?pv <pred:unit> \"1\" . ?pv <pred:value> \"104072\"^^xsd:double . 
#     [ <pred:fact_h> ?e ; <pred:fact_r> <population> ; <pred:fact_t> ?pv ] <point_in_time> ?qpv .  }
# """ 
# sentence = "When did the big city whose postal code is 54000 have a population of 104072?"

"""
SELECT DISTINCT ?e WHERE { 
    ?e <pred:instance_of> ?c . ?c <pred:name> \"town\" . 
    ?e <TOID> ?pv . ?pv <pred:value> \"4000000074573917\" . 
    ?e <OS_grid_reference> ?pv_1 . ?pv_1 <pred:value> \"SP8778\" .  }
"""
sentence = "Which town has a TOID of 4000000074573917 and has an OS grid reference of SP8778?"


td = tokenizer(sentence, return_tensors='pt')
pred_ids = raw_model.generate(
    inputs=td.input_ids, max_length=300, num_beams=1,do_sample=False
)
print(tokenizer.batch_decode(pred_ids, clean_up_tokenization_spaces=True, skip_special_tokens=True))

['SELECT DISTINCT?e WHERE {?e <pred:instance_of>?c.?c <pred:name> "town".?e <TOID>?pv.?pv <pred:value> "4000000074573917".?e <OS_grid_reference>?pv_1.?pv_1 <pred:value> "SP8778".  }']


# Bart With copy mechanism

In [21]:


td = tokenizer(sentence, return_tensors='pt')
pred_ids = copy_model.generate(
    inputs=td.input_ids, enable_copy=True, output_attentions=True, output_hidden_states=True,
    max_length=300
)
print(tokenizer.batch_decode(pred_ids,clean_up_tokenization_spaces=True, skip_special_tokens=True))

['SELECT D D WHERE <pred>?_1. <TOID>?pv".?e <OS_grid_reference>?pv_?pv.?pv_1 <pred DISTINCT?pv.?pv <pred DISTINCT?e WHERE {?e <pred DISTINCT?pv.?pv <pred DISTINCT? DISTINCT?pv.?pv < DISTINCT? }']
